In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Spark").getOrCreate()

# Capítulo 2

##### 1. Aplicar no solo count (para obtener el número de líneas) y show sino probar distintas sobrecargas del método show (con/sin truncate, indicando/sin indicar num de filas, etc) así como también los métodos, head, take, first (diferencias entre estos 3?)

In [20]:
quijote = spark.read.text("data/el_quijote.txt")

In [21]:
print(f"Número de lineas del archivo el quijote: {quijote.count()}")

Número de lineas del archivo el quijote: 2186


In [22]:
quijote.show()

+--------------------+
|               value|
+--------------------+
|DON QUIJOTE DE LA...|
|Miguel de Cervant...|
|                    |
|       PRIMERA PARTE|
|CAPÍTULO 1: Que ...|
|En un lugar de la...|
|Tuvo muchas veces...|
|En resolución, e...|
|historia más cie...|
|Decía él, que e...|
|En efecto, remata...|
|Imaginábase el p...|
|linaje y patria, ...|
|Limpias, pues, su...|
|Capítulo 2: Que ...|
|Hechas, pues, est...|
|Estos pensamiento...|
|Con estos iba ens...|
|Autores hay que d...|
|muertos de hambre...|
+--------------------+
only showing top 20 rows



In [31]:
quijote.show(10)

+--------------------+
|               value|
+--------------------+
|DON QUIJOTE DE LA...|
|Miguel de Cervant...|
|                    |
|       PRIMERA PARTE|
|CAPÍTULO 1: Que ...|
|En un lugar de la...|
|Tuvo muchas veces...|
|En resolución, e...|
|historia más cie...|
|Decía él, que e...|
+--------------------+
only showing top 10 rows



Diferencias entre head, take, first:
- Head : De forma predefinida te muestra las 20 primeras lineas
- Take : Es necesario introducir un número de líneas
- First : Solo te muestra la primera linea, no admite argumentos.

##### 2. Del ejercicio de M&M aplicar: 

In [43]:
mm_df = spark.read.format("csv").option("header","true").load("data/mm.csv")

In [45]:
mm_df.show(2)

+-----+-----+-----+
|State|Color|Count|
+-----+-----+-----+
|   TX|  Red|   20|
|   NV| Blue|   66|
+-----+-----+-----+
only showing top 2 rows



##### Otras operaciones de agregación como el Max con otro tipo de ordenamiento (descendiente).

In [52]:
from pyspark.sql.functions import *
mm_df.groupby("State","Color").agg(max("Count").alias("Max"),min("Count").alias("Min"),avg("Count").alias("Media")).orderBy(desc("Media")).show()

+-----+------+---+---+------------------+
|State| Color|Max|Min|             Media|
+-----+------+---+---+------------------+
|   CO| Brown| 99| 10| 56.57729468599034|
|   WY| Brown| 99| 10| 56.20757180156658|
|   NM|   Red| 99| 10| 56.03491124260355|
|   TX|Orange| 99| 10|55.880750605326874|
|   WA|Yellow| 99| 10|  55.8749248346362|
|   CA|Yellow| 99| 10|  55.8693967902601|
|   WA|   Red| 99| 10| 55.85397965290245|
|   NV| Brown| 99| 10| 55.81050090525045|
|   WA| Brown| 99| 10| 55.77112043139604|
|   CA| Brown| 99| 10|55.740395809080326|
|   WY| Green| 99| 10|55.657227138643066|
|   CA|  Blue| 99| 10| 55.59762944479102|
|   UT| Green| 99| 10|55.557510999371466|
|   NV|   Red| 99| 10|  55.4944099378882|
|   CO|Orange| 99| 10|55.402557856272836|
|   NM| Brown| 99| 10|55.392412566686424|
|   WA|  Blue| 99| 10|55.314461538461536|
|   TX|   Red| 99| 10|55.306666666666665|
|   TX| Brown| 99| 10| 55.29311395490554|
|   CA|   Red| 99| 10| 55.26992753623188|
+-----+------+---+---+------------

##### Hacer un ejercicio como el “where” de CA que aparece en el libro pero indicando más opciones de estados (p.e. NV, TX, CA, CO).

In [66]:
mm_df.where(col("State").isin(["NV","TX","CA","CO"])).groupby("State","Color").agg(max("Count").alias("Max"),min("Count").alias("Min"),avg("Count").alias("Media")).orderBy(desc("State")).show(25)

+-----+------+---+---+------------------+
|State| Color|Max|Min|             Media|
+-----+------+---+---+------------------+
|   TX|  Blue| 99| 10|54.811648079306075|
|   TX|   Red| 99| 10|55.306666666666665|
|   TX| Brown| 99| 10| 55.29311395490554|
|   TX|Yellow| 99| 10| 55.09042865531415|
|   TX|Orange| 99| 10|55.880750605326874|
|   TX| Green| 99| 10| 55.12550374208405|
|   NV|Orange| 99| 10|54.865070093457945|
|   NV| Brown| 99| 10| 55.81050090525045|
|   NV|   Red| 99| 10|  55.4944099378882|
|   NV| Green| 99| 10| 53.78739693757362|
|   NV|Yellow| 99| 10|54.561194029850746|
|   NV|  Blue| 99| 10|53.797369994022716|
|   CO| Brown| 99| 10| 56.57729468599034|
|   CO|  Blue| 99| 10| 55.11032448377581|
|   CO|   Red| 99| 10|55.089285714285715|
|   CO| Green| 99| 10| 54.71336835960304|
|   CO|Yellow| 99| 10| 55.22254503195816|
|   CO|Orange| 99| 10|55.402557856272836|
|   CA|  Blue| 99| 10| 55.59762944479102|
|   CA|Yellow| 99| 10|  55.8693967902601|
|   CA|   Red| 99| 10| 55.26992753

In [93]:
mm_df.createOrReplaceTempView("mm")
spark.sql("SELECT Color, avg(Count) as Media FROM mm GROUP BY Color").show()

+------+------------------+
| Color|             Media|
+------+------------------+
|Orange| 54.84775708211056|
| Green|54.711129489603024|
|  Blue| 54.86752994102985|
| Brown|55.463537250151425|
|Yellow| 54.95409621338414|
|   Red| 55.16962512786569|
+------+------------------+



In [81]:
spark.sql("SHOW TABLES").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        |       mm|       true|
+--------+---------+-----------+



In [96]:
mm_df.printSchema()

root
 |-- State: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Count: string (nullable = true)



# Capitulo 3

##### 1. Leer el CSV del ejemplo del cap2 y obtener la estructura del schema dado por defecto.

In [101]:
mm_df = spark.read.format("csv").option("header","true").option("inferSchema","true").load("data/mm.csv")

In [102]:
mm_df.printSchema()

root
 |-- State: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Count: integer (nullable = true)



##### 2. Cuando se define un schema al definir un campo por ejemplo StructField('Delay', FloatType(), True) ¿qué significa el último parámetro Boolean?

El tercer argumento nos permite la posibilidad de que se permita introducir valores nulos en el campo. 

##### 3. Dataset vs DataFrame (Scala). ¿En qué se diferencian a nivel de código?

Una de la diferencias entre los Datasets y los DataFrame son los tipados en los campos. Los Datasets tiene un fuerte tipado

##### 4.Utilizando el mismo ejemplo utilizado en el capítulo para guardar en parquet y guardar los datos en los formatos

In [104]:
fire_call = spark.read.format("csv").option("header","true").option("inferSchema","true").load("data/sf-fire-calls.csv")

In [106]:
# Save formato parquet
fire_call.write.parquet("data/fire_call.parquet")
# Save formato avro
fire_call.write.format("avro").save("data/fire_call.avro")
# Save formato csv
fire_call.write.csv("data/fire_call.csv")

# Capitulo 4 

##### 1. GlobalTempView vs TempView

La diferencia es la accesibilidad, mediante el comando GlobalTempView puedes acceder a la view en las diferentes sessiones activas mientras que con las Tempview no.

##### 2.Leer los AVRO, Parquet, JSON y CSV escritos en el cap3

In [111]:
# Load formato parquet
fire_parquet = spark.read.parquet("data/fire_call.parquet")
# Load formato avro
fire_avro = spark.read.format("avro").load("data/fire_call.avro")
# Load formato csv
fire_csv = spark.read.format("csv").load("data/fire_call.csv")

# Capitulo 4

##### 1. Pros y Cons utilizar UDFs

Nos permite serializar las funciones del lenguaje python para poder utilizarlas en Spark pero tiene problemas de rendimiento comparado con Spark SQL ya que debe mandar la informacion de los JVM a Python.

##### 2.Instalar MySQL, descargar driver y cargar datos de BBDD de empleados

In [117]:
# Cargar con spark datos de empleados, departamentos y salario
empleados = spark.read\
                .format("jdbc")\
                .option("url", "jdbc:mysql://localhost:3306/employees")\
                .option("driver", "com.mysql.jdbc.Driver")\
                .option("dbtable", "employees")\
                .option("user", "root")\
                .option("password", "lutecio")\
                .load()
departamentos = spark.read\
                .format("jdbc")\
                .option("url", "jdbc:mysql://localhost:3306/employees")\
                .option("driver", "com.mysql.jdbc.Driver")\
                .option("dbtable", "departments")\
                .option("user", "root")\
                .option("password", "lutecio")\
                .load()

salario = spark.read\
                .format("jdbc")\
                .option("url", "jdbc:mysql://localhost:3306/employees")\
                .option("driver", "com.mysql.jdbc.Driver")\
                .option("dbtable", "salaries")\
                .option("user", "root")\
                .option("password", "lutecio")\
                .load()
titulo = spark.read\
                .format("jdbc")\
                .option("url", "jdbc:mysql://localhost:3306/employees")\
                .option("driver", "com.mysql.jdbc.Driver")\
                .option("dbtable", "titles")\
                .option("user", "root")\
                .option("password", "lutecio")\
                .load()

In [127]:
empleados.join(salario, empleados.emp_no ==salario.emp_no).join(titulo,empleados.emp_no==titulo.emp_no).show()

+------+----------+----------+---------+------+----------+------+------+----------+----------+------+----------------+----------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|emp_no|salary| from_date|   to_date|emp_no|           title| from_date|   to_date|
+------+----------+----------+---------+------+----------+------+------+----------+----------+------+----------------+----------+----------+
| 10206|1960-09-19|  Alassane|  Iwayama|     F|1988-04-19| 10206| 40000|1988-04-19|1989-04-19| 10206|Technique Leader|1988-04-19|9999-01-01|
| 10206|1960-09-19|  Alassane|  Iwayama|     F|1988-04-19| 10206| 43519|1989-04-19|1990-04-19| 10206|Technique Leader|1988-04-19|9999-01-01|
| 10206|1960-09-19|  Alassane|  Iwayama|     F|1988-04-19| 10206| 46265|1990-04-19|1991-04-19| 10206|Technique Leader|1988-04-19|9999-01-01|
| 10206|1960-09-19|  Alassane|  Iwayama|     F|1988-04-19| 10206| 46865|1991-04-19|1992-04-18| 10206|Technique Leader|1988-04-19|9999-01-01|
| 10206|1960-

##### 3.Diferencia entre Rank y dense_rank (operaciones de ventana)
Ambas funciones sirven para clasificar un registro dentro de un grupo de filas, la diferencia que en caso de que dos registros tengan la misma posición con la función Dense_rank saltara los puestos para el siguiente.